# 장르 속성을 이용한 영화 콘텐츠 기반 필터링
- TMDB 5000 데이터 셋: 영화 데이터 정보 사이트 IMDB.com의 영화 중 주요 영화 5000개에 대한 메타 정보를 가공해서 kaggle에서 제공하는 데이터 셋
- https://www.kaggle.com/tmdb/tmdb-movie-metadata

## 데이터 로딩 및 가공

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

movies = pd.read_csv('datasets/tmdb_5000_movies.csv')
print(movies.shape)
display(movies.head(2))

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


- 분석에 사용할 주요 컬럼 추출
    - id, title, genres, vote_average(평균 평점), vote_count(평점 투표수), popularity(영화 인기도), keywords, overview(영화 개요)

In [2]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [3]:
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."


In [4]:
movies_df['genres'].values[0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

- eval() 과  literal_eval() 의 차이
    - eval()은 문자형태로 되어있는 표현식을 실행하는 함수, 함수나 객체도 실행가능
    - literal_eval()은 파이썬에서 제공하는 기본 데이터 타입 정도만 변환해주는 용도로 사용 가능

- literal_eval() 함수를 통해 genres 와 keywords 컬럼의 값을 List 객체로 변환

In [5]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [6]:
movies_df['genres'].values[0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

- 장르와 키워드 컬럼의 값 중 name 키 값만 원소로 추출하여 리스트로 생성

In [7]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])

In [8]:
movies_df['genres'].values[0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

## 장르 콘텐츠 유사도 측정
- 문자열로 변환된 'genres' 컬럼은 카운트 기반으로 피처 벡터화 변환 (사이킷런의 CountVectorizer 클래스 이용)
- 'genres' 문자열을 피처 벡터화 행렬로 변환한 데이터셋을 코사인 유사도를 통해 비교
- 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다.


- **CountVectorizer**
    - 텍스트에서 단위별 출현 횟수를 카운팅하여 수치 벡터화한다.

In [12]:
# CountVectorizer 실습
from sklearn.feature_extraction.text import CountVectorizer

# ngram_range=(1,1) 단어를 어떻게 끊어올건지 정함 
# 'hello wowrld python thanks' 
vectorizer = CountVectorizer(ngram_range=(1,1))
vectorizer.fit(['첫번째 문서 테스트','두번째 문서 테스트'])

# 학습한 문서의 고유한 어휘가 딕셔너리처럼 각각의 인덱스를 가지고 있다.
print(sorted(vectorizer.vocabulary_.items(), key=lambda x : x[1]))

# # transform(): 문자열 목록을 전달하여 미리 학습 해놓은 사전을 기반으로 어휘의 빈도수를 세어준다.
counts = vectorizer.transform(['직접 첫번째 테스트 두번째 테스트'])
print(counts.toarray())

[('두번째', 0), ('문서', 1), ('첫번째', 2), ('테스트', 3)]
[[1 0 1 2]]


In [13]:
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : ' '.join(x))

In [14]:
movies_df['genres_literal'][0]

'Action Adventure Fantasy Science Fiction'

In [17]:
# min_df : 전체 문서에서 낮은 빈도수를 가지는 단어 피처를 제외하기 위한 파라미터
# 만약, min_df=2 일 경우 전체 문서에 걸쳐 2번 미만으로 나타나는 단어는 피처 추출에서 제외
# ngram_range : 모델의 단어 순서를 보강하기 위한 범위(범위 최솟값, 범위 최댓값)
# (1,1) : 단어를 1개씩 피처로 추출
# (1,2) : 토큰화된 단어를 1개씩, 그리고 순서대로 2개씩 묶어서 피처로 추출
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)  # 4803 = 영화의 개수, 276 = 카운트된 장르의 어휘 개수
print(genre_mat.toarray()[0])

(4803, 276)
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [19]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:3])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]]


In [23]:
genre_sim_sortes_idx = genre_sim.argsort()[:, ::-1]

# 첫 번째 영화와 장르 유사도가 높은 top10 영화의 movies_df 상에서 index 값
# 앞의 0 인덱스는 자기 자신의 인덱스를 가리킨다
print(genre_sim_sortes_idx[0][:10])

[   0 3494  813  870   46   14 1296 1652  419  420]


## 장르 콘텐츠 필터링을 통한 영화 추천

In [24]:
# title_name : 추천 기준이 되는 영화 제목(해당 영화와 장르 유사도가 높은 영화 추천)
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    target_movie = df[df['title'] == title_name]
    title_index = target_movie.index.values
    
    similar_indexes = sorted_idx[title_index, :top_n]
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

- 매우 낮은 평점의 영화도 추천 영화로 선택되는 문제 발생
- 더 많은 후보군을 선정한 뒤에 **영화의 평점에 따라 필터링**하는 방식으로 변경 필요

In [26]:
similar_movies = find_sim_movie(movies_df, genre_sim_sortes_idx, 'The Godfather', 10)
similar_movies[['title','vote_average']]
# 평점 낮아도 출력됨 + 평점 인원의 수

,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [28]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


 - 왜곡된 평점 데이터를 피하기 위해 평점에 평가 횟수를 반영한 가중 평점 방식 적용
 - 가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
     >- v : 개별 영화에 평점을 투표한 횟수
     >- m : 평점을 부여하기 위한 최소 투표 횟수(전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정함 <임의 기준>)
     >- R : 개별 영화에 대한 평균 평점
     >- C : 전체 영화에 대한 평균 평점

In [30]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print(f'C:{C:.3f}  m:{m:.3f}')

C:6.092  m:370.200


In [31]:
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m)) * R) + ((m/(v+m)) * C)

# 가중치 평점 데이터 생성
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [32]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [35]:
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_indexes = sorted_idx[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기준이 되는 영화 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]


In [36]:
similar_movies = find_sim_movie(movies_df, genre_sim_sortes_idx, 'The Godfather', 10)
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
